In [6]:
from experiment import PortfolioModelingExperiment, TrainingConfig
from model import make_model
import pandas as pd

#### Pull Input Data

In [7]:
return_df = pd.read_parquet("return_df.parquet")
start = 2000
cutoff = 500
training_df = return_df[start:-cutoff]
validation_df = return_df[-cutoff:]

n_assets = len(return_df.columns)

#### Create Training Configuration

In [ ]:
# example tranining config

training_config = TrainingConfig(
    batch_size=32,
    n_assets=n_assets,
    tau=41,
    h=4,
    N=4,
    d_model=128,
    dropout=0.1,
    conv_kernel_size=3,
    padding=1,
    learning_rate=0.05,
    epochs=200,
    optimizer="SGD",
    seed=2024,
    data_type="etf",
    model_type="lstm",
)

In [ ]:
model = training_config.make_model()

experiment = PortfolioModelingExperiment(
    model=model,
    training_config=training_config,
    training_data=training_df,
    validation_data=validation_df,
    early_stopping_patience=10,
)

In [ ]:
print(f"experiment hash: {experiment.hash_dir}")

#### Run the Experiment

In [ ]:
experiment.run(validate=True)

### Iterate through hyperparameters

In [ ]:
import itertools as it

batch_sizes = [32, 64, 128, 256]
learning_rates = [0.05, 0.01, 0.005]
taus = [141, 121, 101, 81, 61, 41]
d_models = [32, 64, 128, 256]
paddings = ["same"]
conv_kernel_sizes = [30, 20, 10, 5, 3]
optimizers = ["SGD"]

configurations = list(
    it.product(
        batch_sizes,
        learning_rates,
        taus,
        d_models,
        paddings,
        conv_kernel_sizes,
        optimizers,
    )
)

for (
    batch_size,
    learning_rate,
    tau,
    d_model,
    padding,
    conv_kernel_size,
    optimizer,
) in configurations:
    training_config = TrainingConfig(
        batch_size=batch_size,
        n_assets=n_assets,
        tau=tau,
        h=4,
        N=4,
        d_model=d_model,
        dropout=0.1,
        conv_kernel_size=conv_kernel_size,
        padding=padding,
        learning_rate=learning_rate,
        epochs=100,
        optimizer=optimizer,
        seed=2024,
        data_type="etf",
        model_type="transformer",
    )
    model = training_config.make_model()
    experiment = PortfolioModelingExperiment(
        model=model,
        training_config=training_config,
        training_data=training_df,
        validation_data=validation_df,
        early_stopping_patience=20,
    )
    print(experiment.hash_dir)
    experiment.run(validate=True)